## prep of class: monomer:palindrome 
#### download and analyse pdb entries, creates dataframes for analysis that can be used in creating a data set for DNA-protein structure ML
##### notes: sequence data should be taken from fasta

In [ ]:
from Bio.PDB import PDBList                  # fetches/saves PDB data
from Bio.PDB.MMCIF2Dict import MMCIF2Dict    # parses data in mmCIF files
import pandas as pd
import os

In [ ]:
# map CIF tokens onto dictionary keys. some tokens are repeated in the asu and the assembly, some are not, 
# the assembly can have fewer instances of entities. also, some are not present in assembly, 
# like title and id, src, etc. so we need to grab some from asu and some from assembly.
# warning: some tokens repeated in asu and assembly have different values in the two files!
# also: not all tokens of interest appear in all entries.

# in definitions below, key is name of field (column) in database (dataframe), and value is the CIF token.
# usually tokens from asu entry are all single element entries -- some exceptions
# 1) 'gene': '_entity_src_gen.pdbx_gene_src_gene'
# 2) 'species': '_entity_src_gen.pdbx_gene_src_scientific_name'

asuTokens = { 'pdbid': '_entry.id',\
            'date': '_pdbx_database_status.recvd_initial_deposition_date',\
            'method': '_exptl.method',\
             'title': '_struct.title',\
            'gene': '_entity_src_gen.pdbx_gene_src_gene',\
             'species': '_entity_src_gen.pdbx_gene_src_scientific_name',\
             'keywords': '_struct_keywords.pdbx_keywords',\
            'text': '_struct_keywords.text'
          }
# all keys from assembly are lists of multiple entries
# the lengths of the following (all from _entity_poly) should be 2 : 
#    1) 'polyid': '_entity_poly.entity_id'
#    2) 'polytype': '_entity_poly.type'
#    3) 'seq': '_entity_poly.pdbx_seq_one_letter_code_can'
#    4) 'polystrand': '_entity_poly.pdbx_strand_id',\
# the lengths of the following (all from _entity) should be AT LEAST 2 and ALL EQUAL:
#    5) 'entityid': '_entity.id'
#    6) 'entitytype': '_entity.type'
#    7) 'descr': '_entity.pdbx_description'
#    8) 'MW': '_entity.formula_weight'
#    9) 'number': '_entity.pdbx_number_of_molecules'

assemblyTokens = {'polyid': '_entity_poly.entity_id',\
                'polytype': '_entity_poly.type',\
                'seq': '_entity_poly.pdbx_seq_one_letter_code_can',\
                  'polystrand': '_entity_poly.pdbx_strand_id',\
                'entityid': '_entity.id',\
                'entitytype': '_entity.type',\
                'descr': '_entity.pdbx_description',\
                'MW': '_entity.formula_weight',\
                'number': '_entity.pdbx_number_of_molecules'
               }
columnNames = list(asuTokens.keys())+list(assemblyTokens.keys())  # combine all keys into one list for dictionary

In [ ]:
# read in list of pdb codes for initial consideration
dfCodes = pd.read_csv('./monomer:palindrome_v1.csv')
dfCodes[ dfCodes[ 'assembly' ] ==2 ]

In [ ]:

pdbCodes = [ c.lower() for c in  list( dfCodes['pdbid' ] ) ]
assemblies = list( dfCodes[ 'assembly' ] )
print(len(pdbCodes),'codes\n',pdbCodes)
print(len(assemblies),'codes\n',assemblies )

In [ ]:
# define data directories. fetch both asu and assembly files, then run check
asuDirectory      = 'asu'
assemblyDirectory = 'assembly'

# run a check on files:
asuList = os.listdir(asuDirectory)
assemblyList = os.listdir(assemblyDirectory)
asuCodes = [ s.split('.')[0] for s in asuList ]
assemblyCodes = [ s.split('-')[0] for s in assemblyList ]

In [ ]:

for code in pdbCodes:
    if not code in asuCodes:
        print(code, 'not in', asuDirectory)
    if not code in assemblyCodes:
        print(code, 'not in', assemblyDirectory)
        
print( len(asuList), 'files in', asuDirectory, 'directory')
print( len(assemblyList), 'files in', assemblyDirectory, 'directory')

In [ ]:
# parse each pair of mmCIF files (one for asu, one for assembly), and map values to dictionaries
dfDict = { k:[] for k in columnNames }      # holds the values for each entry
lengthDict = { k:[] for k in columnNames }  # holds counts of each value (length of list)

# create the dictionaries of the values I want from the asu header and the bio-assembly header
for pdbCode,assembly in zip(pdbCodes,assemblies):
    print(pdbCode,': START RECORD ---',end =' ')
    asucif       = MMCIF2Dict(asuDirectory+'/'+pdbCode+'.cif')
    assemblycif  = MMCIF2Dict(assemblyDirectory+'/'+pdbCode+'-assembly'+str(assembly)+'.cif')
    for k,v in asuTokens.items():          # iterate over asu token/value pairs
        try:
            value = asucif[v]
        except:
            value = []
        dfDict[k].append(value)
        lengthDict[k].append(len(value))
    for k,v in assemblyTokens.items():      # iterate over bio-assembly token/value pairs
        try:
            value = assemblycif[v]
        except:
            value = []
        dfDict[k].append(value)
        lengthDict[k].append(len(value))
    print('END RECORD.')

In [ ]:
# create dataframes and 
df=pd.DataFrame(dfDict)
lengthDf=pd.DataFrame(lengthDict)

In [ ]:
df

In [ ]:
lengthDf

In [ ]:
# which fields have multiple numbers of entries? This code will list each column label and a set of
# possible values for the lengths of the entries. e.g., pdbid should have just {1}, but gene could have more.
# expectations for monomer:palindrome structures --- see cell near top
# note: polyid, polytype, seq, polystrand should all =2
# note: entityid, entitytype, descr, MW, number should all be equal and >=2

for k in lengthDf.columns:
    print(k,'\t',set(lengthDf[k]))

In [ ]:
# examine these variations 
lengthDf.hist(['gene','species','polyid','polytype','seq','polystrand'])

### now let's find structures to exclude

In [ ]:
# first check for _entity_poly values not =2 
examineColumns = [ 'polyid',\
                 'polytype',\
                  'seq',\
                'polystrand',\
                 ]
dfExclude = {}
for k in examineColumns:
    dfExclude[k] = df[ lengthDf[k]!=2 ]
    print(dfExclude[k]['pdbid'])

In [ ]:
# now check for _entity values  <2 and for equality of length of all _entity values
examineColumns = [ 'entityid',\
                 'entitytype',\
                  'descr',\
                'MW',\
                  'number',\
                 ]
dfExclude = {}
for k in examineColumns:
    dfExclude[k] = df[ lengthDf[k]<2 ]
    print(dfExclude[k]['pdbid'])

dfExclude = {}
for k in examineColumns:
    for kk in examineColumns:
        dfExclude[(k,kk)] = df[ lengthDf[k] != lengthDf[kk] ]
        print('error: ',k,kk,'\n',dfExclude[(k,kk)]['pdbid'])

In [ ]:
# now let's double check for number of chains in assembly.This can be done by looking at polystrand

examineList=[]
removeIndex=[]
for index,item in zip(df.index,df['polystrand']):
    itemsplit = [s.split(',') for s in item]
    itemlength = [len(i) for i in itemsplit]
    if itemlength != [1,2] and itemlength != [2,1]:
        print(index, df.iloc[index]['pdbid'],itemlength)
        examineList.append(df.iloc[index]['pdbid'][0].lower() )
        removeIndex.append(index )


In [ ]:
removeIndex